In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona

In [3]:
df = pd.read_csv('../../../data/emissions_per_hectare.csv')
df.head()

,Country,defor_techden,refor_techden,formgmts_techden,gfire_techden,peat_techden,mangrove_techden,mangrest_techden
0,Angola,219.6426,179.0867,4.1586,18.1093,NaN,1404.4666,432.816400
1,Anguilla,136.5638,NaN,0.0000,NaN,NaN,1215.9538,5095.900000
2,Antigua & Barbuda,136.5638,NaN,0.0000,NaN,NaN,1342.5897,714.171100
3,Aruba,136.5638,NaN,0.0000,NaN,NaN,1268.6633,577.412800
4,Australia,NaN,238.2483,17.0699,5.6156,1263.144,1280.0382,436.696045


In [4]:
df_long = pd.melt(df, id_vars='Country')
df_long

,Country,variable,value
0,Angola,defor_techden,219.6426
1,Anguilla,defor_techden,136.5638
2,Antigua & Barbuda,defor_techden,136.5638
3,Aruba,defor_techden,136.5638
4,Australia,defor_techden,NaN
...,...,...,...
737,Vietnam,mangrest_techden,368.3457
738,United States Virgin Islands,mangrest_techden,505.6085
739,Yemen,mangrest_techden,756.2951
740,Zambia,mangrest_techden,NaN


In [5]:
df_long.variable.unique()

array(['defor_techden', 'refor_techden', 'formgmts_techden',
       'gfire_techden', 'peat_techden', 'mangrove_techden',
       'mangrest_techden'], dtype=object)

In [6]:
df_vars = pd.DataFrame(data={'variable':['defor_techden', 'refor_techden', 'formgmts_techden',
       'gfire_techden', 'peat_techden', 'mangrove_techden',
       'mangrest_techden'],
       'category':['Reduce deforestation','Reforestation (Tropics)','Forest Management (Global)','Grassland and savanna fire mgmt',
       'Reduce peatland degradation and conversion','Reduce mangrove loss','Mangrove Restoration'],
       'mangrove related': ['no','no','no','no','no','yes','yes']})
df_vars

,variable,category,mangrove related
0,defor_techden,Reduce deforestation,no
1,refor_techden,Reforestation (Tropics),no
2,formgmts_techden,Forest Management (Global),no
3,gfire_techden,Grassland and savanna fire mgmt,no
4,peat_techden,Reduce peatland degradation and conversion,no
5,mangrove_techden,Reduce mangrove loss,yes
6,mangrest_techden,Mangrove Restoration,yes


## Add location ids

In [24]:
rp = df[['Country']]
rp.head()

,Country
0,Angola
1,Anguilla
2,Antigua & Barbuda
3,Aruba
4,Australia


In [21]:
# Import gadm file to retrieve ISO codes
gadm = gpd.read_file('../../../data/gadm36_level0_original/gadm36_level0_original.shp')
isocodes = gadm[['GID_0', 'NAME_0']]

In [22]:
# Import locations to get staging ids
locations = pd.read_csv('../../../data/staging_locations.csv')
locations = locations[locations['location_type'] == 'country']
locations = locations[['id', 'name', 'iso']].copy()
locations.head()

,id,name,iso
159,1402,Dominican Republic,DOM
160,1401,Colombia,COL
161,1400,"Congo, DRC",COD
162,1399,Australia,AUS
163,1398,Angola,AGO


In [27]:
country_iso = pd.merge(rp, isocodes, how='left', left_on='Country', right_on='NAME_0')
country_iso.rename(columns={'GID_0': 'iso'}, inplace=True)
country_iso.head()

,Country,iso,NAME_0
0,Angola,AGO,Angola
1,Anguilla,AIA,Anguilla
2,Antigua & Barbuda,NaN,NaN
3,Aruba,ABW,Aruba
4,Australia,AUS,Australia


In [31]:
country_iso_locid = pd.merge(country_iso, locations, how='left', on='iso').drop(columns=['NAME_0', 'name'])
country_iso_locid

,Country,iso,id
0,Angola,AGO,1398.0
1,Anguilla,AIA,NaN
2,Antigua & Barbuda,NaN,NaN
3,Aruba,ABW,NaN
4,Australia,AUS,1399.0
...,...,...,...
101,Vietnam,VNM,1364.0
102,United States Virgin Islands,NaN,NaN
103,Yemen,YEM,1366.0
104,Zambia,ZMB,NaN


## Final format

In [34]:
df_complete = pd.merge(df_long, country_iso_locid, how='left', on='Country').drop(columns=['Country', 'iso'])
df_complete['unit'] = 'tCO2eq'
df_complete.rename(columns={'variable': 'indicator', 'id': 'location_id'}, inplace=True)
df_complete = df_complete[df_complete['location_id'].isna() == False]
df_complete

,indicator,value,location_id,unit
0,defor_techden,219.6426,1398.0,tCO2eq
4,defor_techden,NaN,1399.0,tCO2eq
6,defor_techden,NaN,1374.0,tCO2eq
7,defor_techden,496.2587,1373.0,tCO2eq
8,defor_techden,443.8740,1376.0,tCO2eq
...,...,...,...,...
734,mangrest_techden,664.8271,1324.0,tCO2eq
735,mangrest_techden,646.0532,1394.0,tCO2eq
736,mangrest_techden,444.5305,1362.0,tCO2eq
737,mangrest_techden,368.3457,1364.0,tCO2eq


In [35]:
df_complete.to_csv('../../../data/mitigation_emission_data.csv', index=False)